In [1]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizer

In [6]:

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [9]:
from PyPDF2 import PdfReader
import nltk
nltk.download('punkt')

# Extracting Text from PDF
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf = PdfReader(file)
        text = " ".join(page.extract_text() for page in pdf.pages)
    return text

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [8]:
text = extract_text_from_pdf("CollieryControlOorder2000.pdf")

FileNotFoundError: [Errno 2] No such file or directory: 'CollieryControlOorder2000.pdf'

In [ ]:
# Step 1: Remove special characters and symbols
cleaned_text = ''.join(e for e in text if (e.isalnum() or e.isspace() or e in ['.', ',', ';', ':', '(', ')']))

# Step 2: Remove extra spaces and line breaks
cleaned_text = ' '.join(cleaned_text.split())

# Step 3: Join lines
cleaned_text = cleaned_text.replace('\n', ' ')

In [ ]:
def process_text_chunk(chunk_text, question, max_seq_length=512):
    tokenized_question = tokenizer.encode(question, add_special_tokens=True, return_tensors="pt")
    
    all_answers = []
    print("len=",len(chunk_text))
    for start in range(0, len(chunk_text), max_seq_length):
        end = start + max_seq_length
        chunk = chunk_text[start:end]
        print("count: ",start)
        tokenized_chunk = tokenizer.encode(chunk, add_special_tokens=True, return_tensors="pt")
        
        input_ids = torch.cat([tokenized_question, tokenized_chunk], dim=1)
        
        output = model(input_ids)
        answer_start = torch.argmax(output.start_logits)
        answer_end = torch.argmax(output.end_logits) + 1
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end]))
        
        # Compute confidence level based on the sum of start and end logits
        confidence = output.start_logits[0][answer_start] + output.end_logits[0][answer_end - 1]
        
        all_answers.append({"answer": answer, "confidence": confidence.item()})
    
    return all_answers

In [ ]:
# Example chunk text and question
chunk_text = cleaned_text
question = "How can a colliery owner obtain permission to open a coal mine, seam, or section of a seam?"

# Process the chunk with the question
answers = process_text_chunk(chunk_text, question)

# Sort answers by confidence level (higher confidence first)
answers.sort(key=lambda x: x["confidence"], reverse=True)

# Print answers with confidence levels
for i, answer in enumerate(answers):
    print(f"Answer {i + 1}: {answer['answer']} (Confidence: {answer['confidence']:.2f})")

In [10]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizer
from PyPDF2 import PdfReader
import nltk
nltk.download('punkt')

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')



# Extracting Text from PDF
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf = PdfReader(file)
        text = " ".join(page.extract_text() for page in pdf.pages)
    return text

text = extract_text_from_pdf("CollieryControlOorder2000.pdf")

# Step 1: Remove special characters and symbols
cleaned_text = ''.join(e for e in text if (e.isalnum() or e.isspace() or e in ['.', ',', ';', ':', '(', ')']))

# Step 2: Remove extra spaces and line breaks
cleaned_text = ' '.join(cleaned_text.split())

# Step 3: Join lines
cleaned_text = cleaned_text.replace('\n', ' ')


def process_text_chunk(chunk_text, question, max_seq_length=512):
    tokenized_question = tokenizer.encode(question, add_special_tokens=True, return_tensors="pt")

    all_answers = []
    print("len=", len(chunk_text))
    for start in range(0, len(chunk_text), max_seq_length):
        end = start + max_seq_length
        chunk = chunk_text[start:end]
        print("count: ", start)
        tokenized_chunk = tokenizer.encode(chunk, add_special_tokens=True, return_tensors="pt")

        input_ids = torch.cat([tokenized_question, tokenized_chunk], dim=1)

        output = model(input_ids)
        answer_start = torch.argmax(output.start_logits)
        answer_end = torch.argmax(output.end_logits) + 1
        answer = tokenizer.convert_tokens_to_string(
            tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end]))

        # Compute confidence level based on the sum of start and end logits
        confidence = output.start_logits[0][answer_start] + output.end_logits[0][answer_end - 1]

        all_answers.append({"answer": answer, "confidence": confidence.item()})

    return all_answers


# Example chunk text and question
chunk_text = cleaned_text
question = "How can a colliery owner obtain permission to open a coal mine, seam, or section of a seam?"

# Process the chunk with the question
answers = process_text_chunk(chunk_text, question)

# Sort answers by confidence level (higher confidence first)
answers.sort(key=lambda x: x["confidence"], reverse=True)

# Print answers with confidence levels
for i, answer in enumerate(answers):
    print(f"Answer {i + 1}: {answer['answer']} (Confidence: {answer['confidence']:.2f})")
    

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model

len= 8175
count:  0
count:  512
count:  1024
count:  1536
count:  2048
count:  2560
count:  3072
count:  3584
count:  4096
count:  4608
count:  5120
count:  5632
count:  6144
count:  6656
count:  7168
count:  7680
Answer 1: in writing of the central government (Confidence: 9.41)
Answer 2: power to inspect collieries (Confidence: 5.65)
Answer 3: require any owner or agent or manager of a colliery to give any information in his possession (Confidence: 5.52)
Answer 4: in accordance with the procedure specified in subclause ( 1 ) (Confidence: 4.89)
Answer 5: cause the owner , agent or manager of a colliery or any person engaged in or incharge of the loading of coal in wagons , trolleys or trucks in a colliery (Confidence: 4.56)
Answer 6: declaration of grades of coal , the same may be referred to the coal controller (Confidence: 4.55)
Answer 7: without the prior permission in writing of the central government (Confidence: 4.45)
Answer 8: the owner , agent or manager of a colliery shall dec